# Rain Prediction for Australia
This project was performed as the Final Project for the Data Scientist certification at Coderhouse academy.

The dataset was extracted from Kaggle: https://www.kaggle.com/datasets/arunavakrchakraborty/australia-weather-data

## Audience
Every person or entity that wants to know if tomorrow will rain or not in Australia. This audience includes agricultural, insurance, food and other companies that need to plan their operations based on the weather.

## Commercial context
Weather services in their free version offer a very vague system: it is based on repetitions of the same climate variables at a similar stage in previous periods.

## Main goal
Determine if it will rain the next day, in different cities in Australia, without relying solely on the repetition of values in the variables.

## Analytical context
There is data on temperature, humidity level, winds, presence of clouds/sun and atmospheric pressure for more than 100.000 days, from different cities in Australia.

## Secondary questions
1. What is the most important variable to predict whether it will rain tomorrow or not? And the 5 most important?
2. Is there any variable that is infallible (>80% correlation) when predicting rain?
3. What is the wind that most influences the probability of rain the next day?
4. What relationship is there between the extreme T° (maximum - minimum) with the time at which the T° are taken?
5. Given that we have some measurements taken at 2 times, the measurements at which times are more important for predicting tomorrow's rain?

In [229]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import matplotlib as mpl
import numpy as np
from scipy import stats
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
from sklearn import metrics
from sklearn.model_selection import learning_curve
import statsmodels.formula.api as sm
from geopy.geocoders import Nominatim
import pypyodbc as podbc

In [ ]:
conn = podbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                     "Server=ACER-SWIFT-3-SF\SQLEXPRESS;"
                     "Database=weatherAUS;"
                     "Trusted_Connection=yes;")

weather = pd.read_sql_query('''SELECT * FROM [dbo].[weather_AUS]''', conn)

conn.close()

In [ ]:
weather.head()

## Exploration section
`Date` - Date of the record

`Location` - City name

`MinTemp` - The minimum temperature during a particular day `[Celsius degrees]`

`MaxTemp` - The maximum temperature during a particular day `[Celsius degrees]`

`Lluvia` - Rain during a particular day `[millimeters]`

`Evaporación` - Evaporation during a particular day `[millimeters]`

`Sunshine` - Bright sun during a particular day `[hours]`

`WindGusDir` - The direction of the strongest wind gust during a particular day `[16 points of the compass]`

`WindGustSpeed` ​​​​- Speed of the strongest wind gust during a particular day `[kilometers per hour]`

`WindDir9am` - Wind direction during the 10 previous minutes to 9 am `[cardinal points]`

`WindDir3pm` - Wind direction during the 10 previous minutes to 3:00 pm `[cardinal points]`

`WindSpeed ​​9am` - Wind speed during the 10 previous minutes to 9 am `[kilometers per hour]`

`WindSpeed3pm` - Wind speed during the 10 previous minutes to 3 pm `[kilometers per hour]`

`Humidity9am` - Wind humidity at 9 am `[percetage]`

`Humidity3pm` - Wind humidity at 3 pm `[percetage]`

`Presión9am` - Atmospheric pressure at 9 am `[hectopascles]`

`Presión3pm` - Atmospheric pressure at 3 pm `[hectopascles]`

`Cloud9am` - portion of the sky obscured by clouds at 9 am `[eighths of sky]`

`Cloud3pm` - portion of the sky obscured by clouds at 3 pm `[eighths of sky]`

`Temp9am` - Temperature at 9am `[Celsius degrees]`

`Temp3pm` - Temperature at 3 pm `[Celsius degrees]`

`RainToday` - If it rains today, then value is 1 `[yes]`. If it does not rain today, then value is 0 `[no]`

`RainTomorrow` - If it rains tomorrow, then value is 1 `[yes]`. If it does not rain tomorrow, then value is 0 `[no]`

The color and letter format for graphics is defined

In [ ]:
colours = ['#f2a73d', '#f76157', '#f6daab', '#dabd7b', '#f04155', '#ff823a', '#f2f26f', '#fff7bd', '#95cfb7', '#f40034',
           '#07f9a2', '#09c184', '#0a8967', '#0c5149', '#0d192b', '#fe1cac', '#820081', '#e4b302', '#e7204e', '#3f2c26']

In [ ]:
axtitle_dict = {'family': 'serif', 'color': 'darkred', 'weight': 'bold', 'size': 12}
axlab_dict = {'family': 'serif', 'color': 'black', 'size': 10}

In [ ]:
weather.isnull().sum()

Since they are very different numbers, we see them as a percentage

In [ ]:
round(weather.isnull().sum()*100/weather.shape[0], 2)

At a micro level, here I face the first challenge: columns with many nulls - some with over 40% even.

On the other hand, the target column has only 2.25% of null values

In [ ]:
import missingno as msno

In [ ]:
msno.matrix(weather)

In [ ]:
weather.info()

In [ ]:
weather.shape

In [ ]:
weather.describe().round(1)

Once that exploration is finished, we designate the date column as date (it was found as text before)

In [ ]:
weather.date = pd.DatetimeIndex(weather.date)

The pressure is indicated in Hectopascals so we will change the units to atmospheric units

1[ATM] = 1013.25 [HECTOPASCALS]

In [ ]:
hp_atm = 1013.25
weather['pressure9am'] = weather['pressure9am'] / hp_atm
weather['pressure3pm'] = weather['pressure3pm'] / hp_atm

> # Feature Engineering: Null values

## Numeric columns

We sort the dataset 'prioritizing' first by city and then by date

In [ ]:
weather = weather.sort_values(by=['location', 'date'], axis=0, ascending=[True, True])

1. Numeric columns containing null/NaN values are as follows:
`mintemp`, `maxtemp`, `rainfall`, `evaporation`, `sunshine`, `windgustspeed`, `windspeed9am`, `windspeed3pm`, `humidity9am`, `humidity3pm`, `pressure9am`, `pressure3pm`, `cloud9am`, `cloud3pm`, `temp9am`, `temp3pm`.

In order to fill the null values of the numerical columns listed previously, an iterative imputation model will be used.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
numeric_features = ['mintemp', 'maxtemp', 'rainfall', 'evaporation', 'sunshine', 'windgustspeed', 'windspeed9am', 'windspeed3pm',
                     'humidity9am', 'humidity3pm', 'pressure9am', 'pressure3pm', 'cloud9am', 'cloud3pm', 'temp9am', 'temp3pm']

pct_nulls = round(weather.loc[:, numeric_features].isnull().sum()*100/weather.loc[:, numeric_features].shape[0], 1)
pct_nulls

Every column with a number of null values greater than 35% will be eliminated since estimating so many null data would input significant bias

In [ ]:
to_delete = pct_nulls[pct_nulls > 35].index.to_list()
to_delete

In [ ]:
weather.shape

In [ ]:
weather.drop(columns=to_delete, inplace=True)

In [ ]:
weather.shape

In [ ]:
numeric_features = [x for x in numeric_features if x not in to_delete]
numeric_features

In [ ]:
round(weather.loc[:, numeric_features].isnull().sum()*100/weather.loc[:, numeric_features].shape[0], 1)

In [ ]:
plt.figure(figsize=(15,6))
cor = weather.loc[:, numeric_features].corr()
ax = sns.heatmap(cor, annot=True, fmt='.1f')
ax.set_title('HEATMAP DE CORRELACIÓN', fontdict=axtitle_dict)
plt.show()

According to the correlations between features, other features will be imputed with an iterative method, separating those that have the greatest correlation with each other

### Iterative Imputer: T°

In [ ]:
temps = ['mintemp', 'maxtemp', 'temp9am', 'temp3pm']
Xnum_temps = weather.loc[:, temps].values

In [ ]:
print('Nulls (before): %d' % sum(np.isnan(Xnum_temps).flatten()))

In [ ]:
imputer = IterativeImputer(sample_posterior=True, imputation_order="ascending", random_state=123)
imputer.fit(Xnum_temps)
Xtrans_temps = imputer.transform(Xnum_temps)

In [ ]:
print('Nulls (after): %d' % sum(np.isnan(Xtrans_temps).flatten()))

In [ ]:
weather.loc[:, temps] = Xtrans_temps
round(weather[temps].isnull().sum()*100/weather[temps].shape[0], 1)

### Iterative Imputer: Wind Speeds

In [ ]:
speeds = ['windgustspeed', 'windspeed9am', 'windspeed3pm']
Xnum_speeds = weather.loc[:, speeds].values

In [ ]:
print('Nulls (before): %d' % sum(np.isnan(Xnum_speeds).flatten()))

In [ ]:
imputer = IterativeImputer(sample_posterior=True, imputation_order="ascending", random_state=123)
imputer.fit(Xnum_speeds)
Xtrans_speeds = imputer.transform(Xnum_speeds)

In [ ]:
print('Nulls (after): %d' % sum(np.isnan(Xtrans_speeds).flatten()))

In [ ]:
weather.loc[:, speeds] = Xtrans_speeds
round(weather[speeds].isnull().sum()*100/weather[speeds].shape[0], 1)

### Iterative Imputer: humidity

In [ ]:
sch = ['humidity9am', 'humidity3pm']
Xnum_sch = weather.loc[:, sch].values

In [ ]:
print('Nulls (before): %d' % sum(np.isnan(Xnum_sch).flatten()))

In [ ]:
imputer = IterativeImputer(sample_posterior=True, imputation_order="ascending",
                           random_state=123)
imputer.fit(Xnum_sch)
Xtrans_sch = imputer.transform(Xnum_sch)

In [ ]:
print('Nulls (after): %d' % sum(np.isnan(Xtrans_sch).flatten()))

In [ ]:
weather.loc[:, sch] = Xtrans_sch
round(weather[sch].isnull().sum()*100/weather[sch].shape[0], 1)

### Iterative Imputer: pressure

In this case 3 variables would be repeated: `mintemp`, `windgustspeed`, `temp9am`.

Since these 3 variables have already been imputed, they will not undergo modifications and will only be used to fill in the other 2

In [ ]:
pressures = ['pressure9am', 'pressure3pm', 'mintemp', 'windgustspeed', 'temp9am']
Xnum_pressures = weather.loc[:, pressures].values

In [ ]:
print('Nulls (before): %d' % sum(np.isnan(Xnum_pressures).flatten()))

In [ ]:
imputer = IterativeImputer(sample_posterior=True, imputation_order="ascending", random_state=123)
imputer.fit(Xnum_pressures)
Xtrans_pressures = imputer.transform(Xnum_pressures)

In [ ]:
print('Nulls (after): %d' % sum(np.isnan(Xtrans_pressures).flatten()))

In [ ]:
weather.loc[:, pressures] = Xtrans_pressures
round(weather[pressures].isnull().sum()*100/weather[pressures].shape[0], 1)

### Iterative Imputer: rainfall

To fill in the `rainfall` values we will use all the variables

In [ ]:
rain = ['rainfall']
Xnum_rain = weather.loc[:, numeric_features].values

In [ ]:
print('Nulls (before): %d' % sum(np.isnan(Xnum_rain).flatten()))

In [ ]:
imputer = IterativeImputer(sample_posterior=True, imputation_order="ascending", random_state=123)
imputer.fit(Xnum_rain)
Xtrans_rain = imputer.transform(Xnum_rain)

In [ ]:
print('Nulls (after): %d' % sum(np.isnan(Xtrans_rain).flatten()))

In [ ]:
weather.loc[:, numeric_features] = Xtrans_rain
round(weather[rain].isnull().sum()*100/weather[rain].shape[0], 1)

Now all the numerical fields included in the list have 0% null values

## Categorical columns

Empty character fields: `windgustdir`, `winddir9am`, `winddir3pm`.

This fields were filled with the frequent values (the one that is repeated the most) for each column.

In [ ]:
winds_directions = ['windgustdir', 'winddir9am', 'winddir3pm']

for col in winds_directions:
  print(weather[col].value_counts())
  print('-------------------------------')

In [ ]:
for col in winds_directions:
  weather[col] = weather[col].fillna(weather[col].value_counts().idxmax())

round(weather[winds_directions].isnull().sum()*100/weather[winds_directions].shape[0], 1)

In the original dataset, every null row for the field `raintoday` is always equal to the value in the `raintommorrow` column, but the previous row (a day before).

I created a function that checks, for each day and each city, if the milimeters registered (`rainfall`) are more than 0. In case they are, the `raintoday` column is filled with 'Yes'.

Then the `raintomorrow` field is populated with the data on the following row as explained before (if the field `raintoday`, the row i+1 is representing that it rained on that particular day, the the row i for the column `raintomorrow`, should be 'yes')

In [ ]:
weather.isnull().sum()

In [ ]:
def check_value(row):
  if row['rainfall'] != 0:
    return 'Yes'
  elif row['raintoday'] == 0:
    return 'No'

weather['raintoday'] = weather['raintoday'].fillna(weather.apply(check_value, axis=1))
weather['raintomorrow'] = weather['raintomorrow'].fillna(weather['raintoday'].shift(-1))

round(weather.isnull().sum()*100/weather.shape[0], 1)

In [ ]:
weather.isnull().sum()

The last row is eliminated since there's no a following row to complete the `raintomorrow` field, and that field can't be empty or with null values.

In [ ]:
weather.shape

In [ ]:
weather.dropna(inplace=True)

In [ ]:
weather.shape

> # Feature Engineering: location

In [ ]:
list_0 = ['PearceRAAF', 'NorfolkIsland', 'Launceston', 'Hobart']

for i in list_0:
  weather = weather.drop(weather[weather['location'] == i].index)

In [ ]:
weather.shape[0]

Every row that has values for the cities of PearceRAAF, Norfolk Island, Launceston and Hobart, are eliminated since those are in continental Australia and data is not reliable.

In [ ]:
windspeed_weather = weather.groupby(['location'])[['windspeed9am', 'windspeed3pm', 'windgustspeed']].mean()
windspeed_weather = windspeed_weather.reset_index()
windspeed_weather.head()

We will reduce the `location` field. Now we have 49 cities and we will make have the 6 states of Australia.

To do so, I create a 'df' dataframe where the city is followed by ", Australia" and then use the OpenStreetMaps 'Nominative' API with the GeoPy library to obtain the latitude and longitude of each city. With this data I classify the different cities in each state. Below we will see the distribution of the cities analyzed in these states (Univariate Analysis section).

In [ ]:
df = pd.DataFrame()
df['ciudades'] = weather['location'].unique()

import re
def separar_palabras(cadena):
  patron = r'(?<!^)(?=[A-Z])'
  palabras = re.sub(patron, ' ', cadena)
  return palabras

df['separada'] = df['ciudades'].apply(separar_palabras)

df['separada'] = df['separada'].apply(lambda ciudad: str(ciudad + ', Australia'))

In [ ]:
def obtener_latitud(ciudad):
  geolocator = Nominatim(user_agent="rain_prediction_AUS", timeout=10)
  location = geolocator.geocode(ciudad)
  if location is None:
    return None
  else:
    return location.latitude

In [ ]:
def obtener_longitud(ciudad):
  geolocator = Nominatim(user_agent="rain_prediction_AUS", timeout=10)
  location = geolocator.geocode(ciudad)
  if location is None:
    return None
  else:
    return location.longitude

In [ ]:
df['lat'] = df['separada'].apply(obtener_latitud)

In [ ]:
df['lon'] = df['separada'].apply(obtener_longitud)

In [ ]:
citys = df['ciudades'].tolist()
latitudes = df['lat'].tolist()
longitudes = df['lon'].tolist()

In [ ]:
diccionario_latitudes = dict(zip(citys, latitudes))
weather['Latitud'] = weather['location'].map(diccionario_latitudes)

diccionario_longitudes = dict(zip(citys, longitudes))
weather['Longitud'] = weather['location'].map(diccionario_longitudes)

In [ ]:
mask_wa = df['lon'] < 129
df.loc[mask_wa, 'State'] = 'Western Australia'

mask_nt = (df['lat'] > -26) & (df['lon'] > 129) & (df['lon'] < 138)
df.loc[mask_nt, 'State'] = 'Northern Territory'

mask_sa = (df['lat'] < -26) & (df['lon'] > 129) & (df['lon'] < 141)
df.loc[mask_sa, 'State'] = 'South Australia'

mask_nsw = (df['lat'] < -29) & (df['lon'] > 141)
df.loc[mask_nsw, 'State'] = 'New South Wales'

mask_q = (df['State'] != 'Western Australia') & (df['State'] != 'Northern Territory') & (df['State'] != 'South Australia') & (df['State'] != 'New South Wales')
df.loc[mask_q, 'State'] = 'Queensland'

Below we can see the distribution of cities for each state by absolute and percetaje values

In [ ]:
print(df['State'].value_counts())
print('---------------------------')
print(round(df['State'].value_counts()*100/df['State'].shape[0], 2))

In [ ]:
estados = df['State'].tolist()
diccionario_estados = dict(zip(citys, estados))
weather['location'] = weather['location'].map(diccionario_estados)

In [ ]:
print(weather['location'].value_counts())
print('----------------------------')
print(round(weather['location'].value_counts()*100/weather['location'].shape[0], 2))

Las proporciones vemos que se mantiene de forma aproximada

> # Feature Engineering: Outliers

In [ ]:
weather.dtypes

In [ ]:
categorical_features = []
for col in weather.columns:
	if weather[col].dtype == 'object':
		categorical_features.append(col)

print(f'Columnas categoricas son {categorical_features}')

In [ ]:
print(f'Columnas numéricas son {numeric_features}')

**Outliers** will be handled with **IQR** method

In [ ]:
weather_num = weather[numeric_features]
weather_num.describe()

In [ ]:
Q1 = weather_num.quantile(0.25)
Q3 = weather_num.quantile(0.75)
IQR = Q3 - Q1

((weather_num < (Q1 - 1.5 * IQR)) | (weather_num > (Q3 + 1.5 * IQR))).any()

In [ ]:
fig = plt.figure(figsize=[24, 16])
fig.suptitle('Boxplots', fontsize=18, fontweight = 'bold')
fig.subplots_adjust(top = 0.92)
fig.subplots_adjust(hspace = 0.4, wspace = 0.23)
for i, col in enumerate(numeric_features):
  ax1 = fig.add_subplot(4, 4, i+1)
  ax1 = sns.boxplot(data = weather, x=col, color= colours[i])
  ax1.set_title(f'{col}', fontdict=axtitle_dict)
  ax1.set_xlabel(f'{col}', fontdict=axlab_dict)

`evaporation` and `rainfall` features has most of it values around or equal to 0. This is expected due to the fact that is less likely that it rains in a given day.

In the features that there's involved the wind speed, there are a lot of outliers.

In [ ]:
diccionario = {}
for col in list(numeric_features):
  percentile25 = weather[col].quantile(0.25)
  percentile75 = weather[col].quantile(0.75)
  IQR  = percentile75 - percentile25
  upper_limit = percentile75 + 1.5 * IQR
  lower_limit = percentile25 - 1.5 * IQR
  diccionario['upper_limit'+ '_' + col] = upper_limit
  diccionario['lower_limit'+ '_' + col] = lower_limit

In [ ]:
for col in list(numeric_features):
    weather[col] = np.where(
        weather[col] > diccionario['upper_limit_' + col],
        diccionario['upper_limit_' + col],
        np.where(
            weather[col] < diccionario['lower_limit_' + col],
            diccionario['lower_limit_' + col],
            weather[col]
       )
   )

In [ ]:
# Verificando
fig = plt.figure(figsize=[24, 16])
fig.suptitle('Boxplot de data corregida', fontsize=18, fontweight = 'bold')
fig.subplots_adjust(top = 0.92)
fig.subplots_adjust(hspace = 0.4, wspace = 0.15)
for i, col in enumerate(list(numeric_features)):
    ax1 = fig.add_subplot(5, 3, i+1)
    ax1 = sns.boxplot(data = weather, x=col, color= colours[i])
    ax1.set_title(f'{col}', fontdict=axtitle_dict)
    ax1.set_xlabel(f'{col}', fontdict=axlab_dict)

> # Exploratory Data Analysis

## Univariate analysis

In [ ]:
count = weather['date'].dt.year.value_counts()

In [ ]:
ax = sns.barplot(x = count.index, y = count.values)
ax.set_title(f'Cant de registros por año', fontdict=axtitle_dict)
ax.set_xlabel('Año', fontdict=axlab_dict)

In [ ]:
fig = plt.figure(figsize=[24, 10])

fig.subplots_adjust(top = 0.92)
fig.subplots_adjust(hspace = 0.25, wspace = 0.23)
for i, columns in enumerate(winds_directions):
  input = np.unique(weather[columns], return_counts = True)
  col= 'input'
  ax1 = fig.add_subplot(1, 3, i+1)
  ax1 = sns.barplot(x=list(eval(f'{col}[0]')), y=list(eval(f'{col}[1]')))
  ax1.set_title(f'{columns}', fontdict=axtitle_dict)
  ax1.set_xlabel(f'{columns}', fontdict=axlab_dict)
  ax1.set_ylabel('Count', fontdict=axlab_dict)

In [ ]:
fig = plt.figure(figsize=[20, 20])
fig.suptitle('Categorical fields - Count Plot', fontsize=18, fontweight = 'bold')
fig.subplots_adjust(top = 0.92)
fig.subplots_adjust(hspace = 0.25, wspace = 0.23)
for i, columns in enumerate(categorical_features):
  input = np.unique(weather[columns], return_counts = True)
  col= 'input'
  ax1 = fig.add_subplot(3, 2, i+1)
  ax1 = sns.barplot(x=list(eval(f'{col}[0]')), y=list(eval(f'{col}[1]')))
  ax1.set_title(f'{columns}', fontdict=axtitle_dict)
  ax1.set_xlabel(f'{columns}', fontdict=axlab_dict)
  ax1.set_ylabel('Count', fontdict=axlab_dict)

Se ve que las clases están desbalanceadas... en especial las variables `raintoday` y `raintomorrow`

In [ ]:
fig = plt.figure(figsize=[24, 18])
fig.suptitle('Numerical fields - distribution', fontsize=18, fontweight = 'bold')
fig.subplots_adjust(top = 0.95)
fig.subplots_adjust(hspace = 0.25, wspace = 0.25)
for i, col in enumerate(numeric_features):
  ax = fig.add_subplot(4, 4, i+1)
  ax = sns.distplot(weather[col], color=colours[i])
  ax.axvline(weather[col].quantile(q=0.25), color = 'green', linestyle = '--', label = '25% Quartile')
  ax.axvline(weather[col].mean(), color = 'red', linestyle = '--', label = 'Mean')
  ax.axvline(weather[col].median(), color = 'black', linestyle = '--', label = 'Median')
  ax.axvline(weather[col].quantile(q=0.75), color = 'blue', linestyle = '--', label = '75% Quartile')
  ax.set_xlabel(f'{col}', fontdict=axlab_dict)
  ax.set_title(f'{col.upper()}', fontdict=axtitle_dict)
  ax.legend(fontsize=10)

Conclusions:

1. In Australia the most common wind is from the West, from the Indian Ocean. This wind is called `Monzon` and generates strong tropical rainfall in the summer, while dry and cold weather in the winter. This generates that the state called 'Western Australia' (15% of the dataset) has 2 opposite weather conditions throughout the year.

2. This makes particular sense when you consider that in the morning, the predominant winds are from the north, and in the afternoon, they come from the southeast. The winds shift across the western front during the 9 am to 15pm hour time span.

3. The feature `rainfall` becomes almost like a binary TRUE or FALSE flag, similar to the `raintoday` feature. Most days indicate that it did NOT rain, which is logical. Therefore, it would be interesting to know what happened on the day following those when it DID rain.

4. The variables that account for cloud cover exhibit very peculiar behavior with nearly impossible jumps, appearing almost uniformly distributed. It seems like a uniform distribution, but with very large jumps that are almost evenly spaced (in values) from one another.

5. The following variables: `rainfall`, `humidity9am`, `humidity3pm`, and `wind speeds` can't be negative. For all the negative values detected (which appear to be in small quantities based on the graphs), they will be replaced with 0 since it represents the minimum possible value for these variables.


In [ ]:
list_no_nulos = ['rainfall', 'humidity9am', 'humidity3pm', 'windgustspeed', 'windspeed9am', 'windspeed3pm']

In [ ]:
for col in list_no_nulos:
  weather[col] = weather[col].apply(lambda x: 0 if x < 0 else x)

In [ ]:
x = windspeed_weather.loc[:, 'location']
y1 = windspeed_weather['windspeed9am']
y2 = windspeed_weather['windspeed3pm']
y3 = windspeed_weather['windgustspeed']

plt.figure(figsize = (15, 8))

plt.plot(x, y2, color = 'darkorange', label = 'WindSpeed at 3pm')
plt.plot(x, y1, color = 'green', label = 'WindSpeed at 9am', linestyle='--')
plt.plot(x, y3, color = 'blue', label = 'WindGustSpeed')

plt.xlabel('Ciudad', fontdict=axlab_dict)
plt.ylabel('Velocidad [km/h]', fontdict=axlab_dict)
plt.title('Promedio de velocidad de vientos por Ciudad', fontdict=axtitle_dict)
plt.legend(fontsize = 10, loc = 'best')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df = weather.copy()
df['Week'] = df['date'].dt.week

tupla = df.groupby(['Week']).mean()['rainfall']
df = pd.DataFrame(tupla)
fig=px.line(data_frame=df, x=df.index, y='rainfall', title='Promedio Semanal de Precipitaciones', labels={'Week': 'Semana ','rainfall': 'Promedio de lluvia [mm] '})
fig.update_layout(paper_bgcolor='#FFFFFF',plot_bgcolor='#FFFFFF')
fig.show()

## Bivariate analysis

Let's see the distribution of cities across Australia

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
folium_hmap = folium.Map(location=[-26, 134], zoom_start=5, tiles="OpenStreetMap")

datos = list(zip(weather['Latitud'], weather['Longitud']))

hm_wide = HeatMap(data = datos, min_opacity=0.8,
                  radius=10, blur=6,  max_zoom=10)

folium_hmap.add_child(hm_wide)

The distance between cities in really big. This can be a problem in the future for this project

In [ ]:
fig = plt.figure(figsize=[24, 10])
fig.suptitle('Bivariate analysis of Winds Direction', fontsize=18, fontweight = 'bold')
fig.subplots_adjust(top = 0.9)
fig.subplots_adjust(hspace = 0.25, wspace = 0.2)
for i, col in enumerate(list(winds_directions)):
  a = fig.add_subplot(1, 3, i+1)
  a = sns.countplot(x = weather[col], ax = a, hue = weather.raintomorrow, color= colours[i])
  a.set_title(col, fontdict=axtitle_dict)
  a.legend(fontsize=15)

There's a highly variable relationship between the incidence of the direction from which the rain comes and whether it ends up raining or not. For example, the directions ENE and SSW have a very similar number of samples in the dataset (as seen in the univariate analysis), but when the wind comes from the SSW direction, it rains 50% more often compared to when it comes from the ENE direction.

In [ ]:
fig = plt.figure(figsize=[24, 16])
fig.suptitle('Bivariate analysis of numerical fields', fontsize=18, fontweight = 'bold')
fig.subplots_adjust(top = 0.9)
fig.subplots_adjust(hspace = 0.4, wspace = 0.3)

for i, col in enumerate(list(numeric_features)):
  a = fig.add_subplot(4, 4, i+1)
  a=sns.boxplot(data = weather, x = 'raintomorrow', y =col, ax=a)
  a.set_title(col, fontdict=axtitle_dict)

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(15, 8))
sns.scatterplot(data=weather, x="maxtemp", y="mintemp", hue="location", alpha=.5, ax=axs[0])
sns.histplot(data=weather, x="location", hue="location", shrink=.9, alpha=.5,
             legend=True, ax=axs[1])
f.tight_layout()

In [ ]:
contingency_table = pd.crosstab(weather['raintoday'], weather['raintomorrow'])
sns.heatmap(contingency_table, annot=True)
plt.title('Martiz de escenarios', fontdict=axtitle_dict)
plt.xlabel('raintomorrow', fontdict=axlab_dict)
plt.ylabel('raintoday', fontdict=axlab_dict)
plt.show()

## Multivariate analysis

In [ ]:
weather['raintoday'] = weather['raintoday'].map({'Yes': 1, 'No': 0})
weather['raintomorrow'] = weather['raintomorrow'].map({'Yes': 1, 'No': 0})

In [ ]:
list_5 = numeric_features + ['raintoday', 'raintomorrow']

correlaciones = round(weather.loc[:, list_5].corr()['raintomorrow'].sort_values(ascending = False)[1:], 2)
correlaciones

In [ ]:
ax = weather.loc[:, list_5].corr()['raintomorrow'].sort_values(ascending = False)[1:].plot(kind = 'barh', color = '#09c184', figsize = (10, 8))
ax.set_title('Correlacion con raintomorrow', fontsize=18, fontweight = 'bold')
ax.axvline(0, color = 'black')
ax.axvline(0.15, color = '#0a8967', linestyle = 'dotted')
ax.axvline(-0.15, color = '#0a8967', linestyle = 'dotted')
plt.show()

In [ ]:
list_1 = ['date', 'Latitud', 'Longitud']

In [ ]:
list_3 = list_1 + categorical_features + list(weather.location.unique())
list_3

In [ ]:
list_3 = list_1 + categorical_features + list(weather.location.unique())
list_6 = [x for x in list(weather.columns) if x not in list_3] + ['raintoday']
list_6

In [ ]:
plt.figure(figsize=(15,6))
cor = weather.loc[:, list_6].corr()
ax = sns.heatmap(cor, annot=True, fmt='.2f')
ax.set_title('HEATMAP DE CORRELACIÓN', fontdict=axtitle_dict)
plt.show()

> # Feature Engineering: Encoding

## One Hot Encoding: location

In [ ]:
one_hot_encoded = pd.get_dummies(weather['location'], drop_first=False)
one_hot_encoded

In [ ]:
weather = pd.concat([weather, one_hot_encoded], axis=1)

## Label Encoder: Vientos

A list is created where all the different directions from the three variables that include wind directions are added. Then, only the initials of each of these are kept, which will indicate whether the wind is coming from the North, South, East, or West.

For each value, we only get the 1st letter

In [ ]:
for col in winds_directions:
  print(weather[col].value_counts())
  print('-------------------------------')

In [ ]:
for i in winds_directions:
  weather[i] = weather[i].apply(lambda dir: dir[0])

for col in winds_directions:
  print(weather[col].value_counts())
  print('-------------------------------')

In [ ]:
series = {}
df_aux = pd.DataFrame()

for col in winds_directions:
    # Obtener el conteo de valores para cada columna
    serie = weather[col].value_counts()
    # Almacenar la Serie en el diccionario
    series[col] = serie

# Imprimir las Series resultantes
for col, serie in series.items():
  df_aux[col] = serie

df_aux

In [ ]:
total_direc = df_aux.sum(axis=1)
orden_direc = total_direc.sort_values(ascending=True)
print(orden_direc)

In [ ]:
list_orden_direc_viento = list(orden_direc.index)
list_orden_direc_viento

Se crea un diccionario agregandole un numero a cada uno de los posibles valores

In [ ]:
diccionario = {direc_viento: i+1 for i, direc_viento in enumerate(list_orden_direc_viento)}
diccionario

In [ ]:
for i in winds_directions:
  weather[i] = weather[i].map(diccionario)

In [ ]:
weather[winds_directions]

> # Different model combinations

Different combinations between class balancing algorithms and classification algorithms will be used.

2 machine learning classification models:
* Logistic regression
* Decision Tree Classifier

3 different options to balance classes:
* No balancear las clases
* ADASYN
* SMOTE


We end up having 6 different combinations. For each of this combinations, changes in the hyper-parameters were tested but on this notebook there's only the best result for each of this combinations.

> ## Logistic Regression

### Feature selection

Before selecting variables, we will reset the index. Why? As we will see below, the largest index remains 145460 despite not having that many records as a result of removing rows

In [ ]:
max(weather.index)

In [ ]:
weather.reset_index(drop=True, inplace=True)

In [ ]:
max(weather.index)

To select the variables, the SequentialFeatureSelector (SFS) method is used.

It is a "wrapper" type technique that seeks to find the best subset of features by evaluating different combinations of features and selecting those that improve the model's performance.

It can be used in combination with any classification model and can be applied for both forward selection and backward selection.

In this case, forward selection will be used because it has shown better results than backward selection method.

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

In [ ]:
sfs = SFS(LogisticRegression(), k_features=12, forward=True, floating=False, scoring = 'accuracy', cv = 5)

In [ ]:
X_lr = weather.drop(['raintomorrow', 'date', 'Longitud', 'Latitud', 'location'], axis=1)
y_lr = weather['raintomorrow']

In [ ]:
sfs.fit(X_lr, y_lr)
variables_elegidas = list(sfs.k_feature_names_)

In [ ]:
variables_elegidas

### Train/test

In [ ]:
X_lr = weather[variables_elegidas]
y_lr = weather['raintomorrow']

In [ ]:
from sklearn.model_selection import train_test_split
X_lr_train, X_lr_test, y_lr_train, y_lr_test = train_test_split(X_lr, y_lr, test_size = 0.3, random_state = 123, shuffle=True, stratify=y_lr)

In [ ]:
X_lr_train.shape, y_lr_train.shape, X_lr_test.shape, y_lr_test.shape

In [ ]:
y_lr.value_counts()[1]/y_lr.value_counts()[0]

In [ ]:
y_lr_train.value_counts()[1]/y_lr_train.value_counts()[0]

In [ ]:
y_lr_test.value_counts()[1]/y_lr_test.value_counts()[0]

### Class balancing - ADASYN

In the feature `raintomorrow`, the classes were unbalanced, which can potentially cause issues in the future.

In this case, the `ADASYN (Adaptive Synthetic Sampling)` technique is used. Unlike other oversampling techniques like `SMOTE (Synthetic Minority Over-sampling Technique)`, `ADASYN` adaptively adjusts to the distribution of the minority class. Specifically, this algorithm works through the following steps:

1. It calculates the difference between the number of instances in the majority class and the minority class (referred to as the class unbalance difference).

2. For each instance in the minority class, it calculates the distance between that instance and its k nearest neighbors from the majority class.

3. It computes the sample weights (or proportions) for each instance in the minority class based on the class unbalance difference and the distances to its neighbors.

4. Synthetic instances are generated for instances in the minority class using the calculated sample weights. This "generation of synthetic instances" is achieved by interpolating features between a minority class instance and its selected k neighbors.

Essentially, ADASYN generates synthetic instances near regions of the minority class that are harder to classify, therefore improving the model's ability to capture patterns and characteristics of the minority class.

In [ ]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(n_neighbors=5, sampling_strategy=.43, random_state = 123)
X_lr_resampled, y_lr_resampled = adasyn.fit_resample(X_lr_train, y_lr_train)

In [ ]:
y_lr_resampled.value_counts()

The amount of values ib the minority class were increased until 43% of the majority class

**Let's see the ML model and classification report using the ADASYN algorithm for the resampling and LOGISTIC REGRESSION algorithm for the predictions**

In [ ]:
model_lr = LogisticRegression(max_iter=10000)
model_lr.fit(X_lr_resampled, y_lr_resampled)

In [ ]:
from sklearn.metrics import classification_report
y_lr_pred = model_lr.predict(X_lr_test)
print(classification_report(y_true=y_lr_test, y_pred=y_lr_pred))

### Class balancing - SMOTE

In this case, we will use the `SMOTE (Synthetic Minority Over-sampling Technique)` technique. It is an oversampling technique that involves generating new synthetic samples for the minority class from existing observations by linear interpolation between the nearest neighbors of the same class.

It works by taking a sample from the minority class to find its nearest neighbors. Then, one of these neighbors is randomly chosen, and a synthetic example is created somewhere between the original example and the chosen neighbor. This way, the size of the minority class is increased by creating new samples that are a combination of existing samples.

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy = .45)
X_lr_resampled, y_lr_resampled = smote.fit_resample(X_lr_train, y_lr_train)

In [ ]:
y_lr_resampled.value_counts()

The amount of values ib the minority class were increased until 45% of the majority class

**Let's see the ML model and classification report using the SMOTE algorithm for the resampling and LOGISTIC REGRESSION algorithm for the predcitions**

In [ ]:
model_lr = LogisticRegression(max_iter=10000)
model_lr.fit(X_lr_resampled, y_lr_resampled)

In [ ]:
from sklearn.metrics import classification_report
y_lr_pred = model_lr.predict(X_lr_test)
print(classification_report(y_true=y_lr_test, y_pred=y_lr_pred))

### No class balancing

In this case there will be no classes balancing.

A cross validation method called Stratified K Fold will be used. It works as follows:
k parts from the dataset are selected for both train and test portions of the dataset, checking there are no repetitions in the samples selected for each

In [ ]:
from sklearn.model_selection import StratifiedKFold

model_lr = LogisticRegression(max_iter=10000)
n_folds = 5
acum_accuracy=0
i=0

kf = StratifiedKFold(n_splits=n_folds)

for train_index, test_index in kf.split(X_lr, y_lr):
  i += 1

  X_train, X_test = X_lr.loc[train_index], X_lr.loc[test_index]
  y_train, y_test = y_lr.loc[train_index], y_lr.loc[test_index]

  model_lr.fit(X_train, y_train)

  accuracy = model_lr.score(X_test, y_test)

  acum_accuracy += accuracy
  accuracy_redondeada = round(accuracy, 5)

  print(f"Accuracy fold n° {i}: {accuracy_redondeada}")

acum_accuracy_redondeada = round(acum_accuracy/n_folds, 5)
print(f"Accuracy promedio: {acum_accuracy_redondeada}")

> ## Decision Tree Classfier

### Feature selection

In this case we will the backwards selection since it has proven better results

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

In [ ]:
sfs = SFS(DecisionTreeClassifier(), k_features=10, forward=False, scoring = 'accuracy', cv = 5)

In [ ]:
X_dtc = weather.drop(['raintomorrow', 'date', 'Longitud', 'Latitud', 'location'], axis=1)
y_dtc = weather['raintomorrow']

In [ ]:
sfs.fit(X_dtc, y_dtc)
variables_elegidas = list(sfs.k_feature_names_)

### Train/test

In [ ]:
X_dtc = weather[variables_elegidas]
y_dtc = weather['raintomorrow']

In [ ]:
X_dtc_train, X_dtc_test, y_dtc_train, y_dtc_test = train_test_split(X_dtc, y_dtc, test_size = .3, random_state = 123, shuffle=True, stratify=y_dtc)

In [ ]:
X_dtc_train.shape, y_dtc_train.shape, X_dtc_test.shape, y_dtc_test.shape

In [ ]:
y_dtc_train.value_counts()

### Class balancing - ADASYN

In [ ]:
adasyn = ADASYN(n_neighbors=5, sampling_strategy=0.4, random_state = 123)
X_dtc_resampled, y_dtc_resampled = adasyn.fit_resample(X_dtc_train, y_dtc_train)

In [ ]:
y_dtc_resampled.value_counts()

The amount of values ib the minority class were increased until 40% of the majority class

**Let's see the ML model and classification report using the ADASYN algorithm for the resampling and DECISION TREE CLASSIFIER algorithm for the predictions**



In [ ]:
clf = DecisionTreeClassifier(random_state=123, criterion = 'entropy', max_depth=10)
model_dtc = clf.fit(X_dtc_resampled, y_dtc_resampled)

In [ ]:
y_dtc_pred = clf.predict(X_dtc_test)
print(classification_report(y_true=y_dtc_test, y_pred=y_dtc_pred))

### Class balancing - SMOTE

In [ ]:
smote = SMOTE(sampling_strategy = .41)
X_dtc_resampled, y_dtc_resampled = smote.fit_resample(X_dtc_train, y_dtc_train)

In [ ]:
y_dtc_resampled.value_counts()

The amount of values ib the minority class were increased until 41% of the majority class

**Let's see the ML model and classification report using the SMOTE algorithm for the resampling and DECISION TREE CLASSIFIER algorithm for the predictions**

In [ ]:
clf = DecisionTreeClassifier(random_state=123, criterion = 'entropy', max_depth=10)
model_dtc = clf.fit(X_dtc_resampled, y_dtc_resampled)

In [ ]:
y_dtc_pred = clf.predict(X_dtc_test)
print(classification_report(y_true=y_dtc_test, y_pred=y_dtc_pred))

### No class balancing

In this case there will be no classes balancing.

A cross validation method called Stratified K Fold will be used. It works as follows:
k parts from the dataset are selected for both train and test portions of the dataset, checking there are no repetitions in the samples selected for each

In [ ]:
clf = DecisionTreeClassifier(random_state=123, criterion = 'entropy', max_depth=10)
n_folds = 7
acum_accuracy=0
i=0

kf = StratifiedKFold(n_splits=n_folds)

for train_index, test_index in kf.split(X_lr, y_lr):
  i += 1

  X_train, X_test = X_lr.loc[train_index], X_lr.loc[test_index]
  y_train, y_test = y_lr.loc[train_index], y_lr.loc[test_index]

  clf.fit(X_train, y_train)

  accuracy = clf.score(X_test, y_test)

  acum_accuracy += accuracy
  accuracy_redondeada = round(accuracy, 5)

  print(f"Accuracy fold n° {i}: {accuracy_redondeada}")

acum_accuracy_redondeada = round(acum_accuracy/n_folds, 5)
print(f"Accuracy promedio: {acum_accuracy_redondeada}")

> # Best model

While in most models there is a problem of low identification of class 1 or 'yes', the main criteria for choosing the model is the 'accuracy' metric.

Since it is a binary classification problem, success is not only when the 'yes' variable is predicted correctly but also when the 'no' variable is predicted correctly, as this implies that the prediction of whether it will rain or not the next day is correct, which is precisely what we are aiming for.

The Decision Tree Classifier (DTC) model is the chosen one using ADASYN as the class balancing method for the target variable. It performs best with the following configuration:
* Input with only 10 variables selected using the SequentialFeatureSelector backwards method.
* Balancing with the ADASYN method using the 5 nearest neighbors and adjusting the class ratio from approximately 0.33 to 0.40.
* The DTC performs best when the criterion used is 'entropy,' with a maximum tree depth of 10.

In [ ]:
adasyn = ADASYN(n_neighbors=5, sampling_strategy=0.4, random_state = 123)
X_dtc_resampled, y_dtc_resampled = adasyn.fit_resample(X_dtc_train, y_dtc_train)

In [ ]:
y_dtc_resampled.value_counts()

In [ ]:
clf = DecisionTreeClassifier(random_state=123, criterion = 'entropy', max_depth=10)
model_dtc = clf.fit(X_dtc_resampled, y_dtc_resampled)

## Matriz de confusión

Let's see the confusion matrix and ROC (receiver operating characteristic curve) for that model

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
matriz = confusion_matrix(y_lr_test, y_lr_pred)
matriz

In [ ]:
cm = confusion_matrix(y_lr_test, y_lr_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicciones')
plt.ylabel('Valores reales')
plt.show()

Note that the testing was conducted on three different models: **Logistic Regression**, **Decision Tree Classifier**, and **Random Forest Classifier**.

While similar results were obtained with all three tested models, the Logistic Regression model showed the best results and consumed the least processing time.

Additionally, different class balancing proportions were tested, and the best result was achieved with a 65% ratio. In other words, a higher proportion of balance improves class detection but worsens prediction accuracy, and vice versa. This indicates that the samples created using the SMOTE method introduce some bias into the prediction. Lowering the balance proportion increases prediction accuracy but reduces effectiveness in class detection.

## Curva ROC

The ROC metric (`Receiver Operating Characteristic`) is used to evaluate and visualize the performance of a binary classification model.
It it a graphic representation of the relation between the `True Positive Rate` and the `False Positive Rate` as the classification threshold is adjusted.

The X-axis of the ROC metric represents the `FPR`, which is the proportion of negative instances incorrectly classified as positive. The Y-axis represents the `TPR`, which is the proportion of positive instances correctly classified as positive. As the classification threshold changes, different pairs of `TPR` and `FPR` are obtained, resulting, each combination, in a point on the ROC graphic.

Ideally, a classification model would have a high `TPR` and a low `FPR`, leading to an ROC curve that approaches the top-left corner of the graph. A larger area under the ROC curve (AUC-ROC) indicates better model performance, where an AUC-ROC **=** 1.0 represents perfect classification, and an AUC-ROC **<=** 0.5 represents random classification.

The ROC curve allows visualizing the performance of a classification model, helping to select the optimal threshold for classification and evaluate the accuracy and balance between true positives and false positives of the model.

In [ ]:
y_dtc_prob = model_dtc.predict_proba(X_dtc_test)[:, 1]

The y_prob matrix has n samples and m classes -> each row represents the possibilty of each class

The "`[:, 1]`" part allow us to obtain the positive class

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(y_dtc_test, y_dtc_prob)

roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='#f04155', lw=2, label='Curva ROC (área = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()

## Conclusions

We observe that using a decision tree performs very well in predicting the value 0, meaning no rain. However, it is not as good at predicting the value 1, on days when it does rain

1. Overall the model is correct 83% of the time, not bad at all

2. The precision is also good, indicating that the model is well-controlled in terms of range

3. Regarding recall, it's found that for the 'NO rain' values (specificity), we have high precision and high recall - something very difficult to achieve. While for the 'YES rain' values (sensitivity), we have good precision but low recall

4. Finally, looking at the more comprehensive metric, we have the F1-score, which gives us a very good value for 'NO rain' but a rather poor one for 'YES rain,' indicating few false values for 0 but many false values for 1. This could be due to class imbalance - see the last cell

5. Main success of the model: 'class 0' of the `raintomorrow` feature (target variable) has high precision and high recall -> the model detects the class well and is very reliable when predicting its values

6. Main problem detected: 'class 1' of the `raintomorrow` feature (target variable) has high precision but low recall -> the model detects the class incorrectly, but when it does, it is very reliable. This could be due to imbalanced test records in the target feature (y_test)

7. The model does not change much with the sample split between train and test. Once again, it was tested from 0.2 to 0.4, and the behavior was approximately the same for all variables analyzed when evaluating (precision, recall, F1-score, the number of correct predictions in the confusion matrix, and the area under the ROC curve)

8. Balancing classes up to nearly equal levels is detrimental here. The only combination where the model improved its performance by synthetically increasing the number of samples of 'class 1' is when it approaches 40%

9. The hyperparameter that has the most impact on the model is the depth allowed for tree creation -> more trees generate better results, with 10 being the optimal depth according to the **'entropy'** measure